In [10]:
from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2021-11-12 22:04:23.314082: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-12 22:04:23.475736: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-12 22:04:23.477050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:0b:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-12 22:04:23.477084: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-11-12 22:04:23.643828: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcubl

In [3]:
TRAINING_DIR = "/root/data/cut_data_limit10000/train/"
VALIDATION_DIR = "/root/data/cut_data_limit10000/validation/"
TEST_DIR = "/root/data/cut_data_limit10000/test/"

In [4]:
training_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='reflect', 
    validation_split=0.2
    )

In [5]:
training_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                          batch_size=128, 
                                                          target_size=(150, 150), 
                                                          class_mode='categorical', 
                                                          subset='training',
                                                         )

Found 51342 images belonging to 3 classes.


In [6]:
validation_generator = training_datagen.flow_from_directory(VALIDATION_DIR, 
                                                          batch_size=128, 
                                                          target_size=(150, 150), 
                                                          class_mode='categorical',
                                                          subset='validation', 
                                                         )

Found 1151 images belonging to 3 classes.


In [7]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

83689472/83683744 [==============================] - 1s 0us/step


NameError: name 'data_augmentation' is not defined

In [8]:
model = Sequential([
        # Conv2D, MaxPooling2D 조합으로 층을 쌓습니다. 첫번째 입력층의 input_shape은 (150, 150, 3)으로 지정합니다.
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2), 
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2), 
        # 2D -> 1D로 변환을 위하여 Flatten 합니다.
        Flatten(), 
        # 과적합 방지를 위하여 Dropout을 적용합니다.
        Dropout(0.5),
        Dense(128, activation='relu'),
        # Classification을 위한 Softmax 
        # 출력층의 갯수는 클래스의 갯수와 동일하게 맞춰줍니다 (3개), activation도 잊지마세요!
        Dense(2, activation='softmax'),
    ])

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

checkpoint_path = f"/root/data/model/cut_data_tmp_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                            save_weights_only=True, 
                            save_best_only=True, 
                            monitor='val_loss', 
                            verbose=1)
epochs=100
history = model.fit(training_generator, 
                    validation_data=(validation_generator),
                    epochs=epochs,
                    callbacks=[checkpoint],
                    )

model.load_weights(checkpoint_path)

# loss = model.evaluate(validation_generator)[0]
# acc = model.evaluate(validation_generator)[1]

model.save(f"/root/data/model/cut_data_model.h5")

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [15]:
from PIL import Image

In [13]:
!pip install pillow

     |████████████████████████████████| 3.1 MB 21.8 MB/s eta 0:00:01
